In [41]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai

In [42]:
import os
from openai import AzureOpenAI

In [43]:
!pip install python-dotenv

In [44]:
from dotenv import load_dotenv

In [45]:
load_dotenv()

True

In [46]:
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("api_version")
deployment = os.getenv("DEPLOYMENT_NAME")

In [47]:
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = api_version

In [48]:
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_version=api_version,
)
      

In [49]:
completion = client.chat.completions.create(
    model=deployment,
    messages= [
    {
      "role": "user",
      "content": "What are the differences between Azure Machine Learning and Azure AI services?"
    }],
    max_tokens=800,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
)
print(completion.to_json())

{
  "id": "chatcmpl-9kmlK9nOFkOzO7DSWtdYZMOO2a98i",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Azure Machine Learning and Azure AI services are both part of Microsoft's suite of cloud-based AI and machine learning tools, but they serve different purposes and target different aspects of the AI lifecycle. Here are the key differences between them:\n\n### Azure Machine Learning\n\n1. **Purpose**: \n   - Azure Machine Learning is designed for building, training, and deploying machine learning models. It's a comprehensive service for data scientists, machine learning engineers, and developers.\n\n2. **Features**:\n   - **Model Training**: Supports a variety of frameworks like TensorFlow, PyTorch, and Scikit-learn.\n   - **Automated ML**: Offers automated machine learning capabilities to automatically build and tune models.\n   - **MLOps**: Provides tools for deploying, monitoring, and managing models in production, supporting

In [50]:
from langchain_openai import AzureChatOpenAI

In [51]:
llm = AzureChatOpenAI(model="gpt-35-turbo")

In [52]:
from langchain_core.messages import AIMessage, HumanMessage

In [53]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [69]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "you are a helpful assistant. Answer all the question to the best of your ability."
    ),
    MessagesPlaceholder(variable_name = "messages"),
]
)

In [70]:
chain =  prompt | llm

In [56]:
resposne = chain.invoke(
    {
       "messages": [

                   HumanMessage(
                       content= "translate this given sentence from english to french: I LOVE AI."
                   ),
                   AIMessage(content= "J'adore la AI."),

                   HumanMessage(content= "what did you just say?"),
       ]    }
)

In [57]:
resposne.content

'I said, "J\'adore la AI" which translates to "I love AI" in English.'

In [60]:
from langchain.memory import ChatMessageHistory

In [61]:
demo_chat_history=ChatMessageHistory()

In [62]:
demo_chat_history.add_user_message("translate this given sentence from english to french: I LOVE AI.")

In [63]:
demo_chat_history.add_ai_message("J'adore la AI.")

In [64]:
demo_chat_history.messages

[HumanMessage(content='translate this given sentence from english to french: I LOVE AI.'),
 AIMessage(content="J'adore la AI.")]

In [65]:
input1="who is a founder of OpenAI?"

In [66]:
demo_chat_history.add_user_message(input1)

In [67]:
demo_chat_history.messages

[HumanMessage(content='translate this given sentence from english to french: I LOVE AI.'),
 AIMessage(content="J'adore la AI."),
 HumanMessage(content='who is a founder of OpenAI?')]

In [71]:
resposne=chain.invoke(
    {
        "messages": demo_chat_history.messages
    }
)



In [72]:
resposne.content

'The co-founders of OpenAI are Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, John Schulman, and Wojciech Zaremba.'

In [73]:
demo_chat_history.add_ai_message(resposne)

In [74]:
input2="what did i ask to you just now?"

In [75]:
demo_chat_history.messages

[HumanMessage(content='translate this given sentence from english to french: I LOVE AI.'),
 AIMessage(content="J'adore la AI."),
 HumanMessage(content='who is a founder of OpenAI?'),
 AIMessage(content='The co-founders of OpenAI are Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, John Schulman, and Wojciech Zaremba.', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 64, 'total_tokens': 103}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'saf

In [76]:
response2 = chain.invoke(
    {
        "messages": demo_chat_history.messages
    }
)

In [77]:
response2.content

'The co-founders of OpenAI are Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, John Schulman, and Wojciech Zaremba.'

In [78]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain2 = prompt2 | llm

In [79]:
prompt3 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt3 | llm


In [80]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [81]:
demo_chat_history2 = ChatMessageHistory()

In [82]:
chain_with_message_history = RunnableWithMessageHistory(
                              chain,
                              lambda session_id: demo_chat_history2,
                              input_message_key = "input",
                              history_message_key ="chat_history",
                              )

In [83]:
chain_with_message_history.invoke(
    {"input": "Translate this sentence from English to French: I love programming."},
    {"configurable": {"session_id": "unused"}}
)

AIMessage(content='Je aime la programmation.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 45, 'total_tokens': 51}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-73b20473-7c31-4597-8e01-692f4f1ad2b5-0', usage_metadata={'input_tokens': 45, 'output_tokens': 6, 'total_tokens': 51})

In [84]:
chain_with_message_history.invoke(
    {"input": "What did I just ask you?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='You just asked me to translate the sentence "I love programming" from English to French.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 334, 'total_tokens': 352}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-89a84029-b1d3-476a-a751-64216400e32c-0', usage_metadata={'input_tokens': 334, 'output_tokens': 18, 'total_tokens': 352})

### Conversational Buffer Memory

In [86]:
from langchain.chains import ConversationChain

In [87]:
conversation = ConversationChain(llm=llm)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [88]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [93]:
question1 = "I am pationate about exploring use cases of Large Language models to solve Business problems"
question2 = "How to analyze business impact of using Large Language Models in Help Desk?"
question3 = "How to track effort reduction due to use of Large Language Models in Customer Support?"

In [94]:
from langchain.chains.conversation.memory import ConversationBufferMemory

In [95]:
conv_buff = ConversationChain(
    llm=llm,
    memory = ConversationBufferMemory()
)

In [96]:
conv_buff("Hello AI")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'input': 'Hello AI',
 'history': '',
 'response': 'Hello! How can I assist you today?'}

In [97]:
from langchain.callbacks import get_openai_callback

In [100]:
def count_tokens(chain,query):
    with get_openai_callback() as cb:
        result = chain.invoke(query)
        print(f"Total Number of Tokens:  {cb.total_tokens}")
    return result

In [101]:
count_tokens(conv_buff, question1)

Total Number of Tokens:  289


{'input': 'I am pationate about exploring use cases of Large Language models to solve Business problems',
 'history': "Human: Hello AI\nAI: Hello! How can I assist you today?\nHuman: I am pationate about exploring use cases of Large Language models to solve Business problems\nAI: That's great to hear! Large language models like GPT-3 have been used in a variety of business applications such as chatbots, content generation, sentiment analysis, and more. They can help businesses automate tasks, improve customer interactions, and gain insights from large amounts of text data. Is there a specific business problem you're interested in exploring with large language models?",
 'response': 'It sounds like you have a strong interest in leveraging large language models for business solutions. There are many potential use cases for these models, such as text summarization, language translation, and even generating code snippets. By utilizing the capabilities of large language models, businesses c

In [103]:
llm2 = AzureChatOpenAI(model="gpt-4o")

In [104]:
conv_buff = ConversationChain(
    llm=llm2,
    memory = ConversationBufferMemory()
)

In [105]:
conv_buff(question1)

{'input': 'I am pationate about exploring use cases of Large Language models to solve Business problems',
 'history': '',
 'response': "That's fantastic! Large Language Models (LLMs) like GPT-3 have a wide range of applications in the business world. Here are a few areas where they can be particularly effective:\n\n1. **Customer Support**: LLMs can power chatbots and virtual assistants to handle customer inquiries, provide information, and even troubleshoot common problems. This not only improves customer satisfaction but also reduces the workload on human support agents.\n\n2. **Content Creation**: Whether it's drafting emails, writing blog posts, or generating marketing copy, LLMs can help create high-quality content quickly. This can be especially useful for maintaining a consistent online presence and engaging with customers through various channels.\n\n3. **Data Analysis and Reporting**: LLMs can analyze large datasets and generate reports in natural language, making complex data 